![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Implementing hybrid search with Redis

Hybrid search is all about combining lexical search with semantic vector search to improve result relevancy. This notebook will cover 3 different hybrid search strategies with Redis:

1. Linear combination of scores from lexical search (BM25) and vector search (Cosine Distance) with the aggregation API
2. Client-Side Reciprocal Rank Fusion (RRF)
3. Client-Side Reranking with a cross encoder model

>Note: Additional work is planed within the Redis core and ecosystem to add more flexible hybrid search capabilities in the future.

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/02_hybrid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Install Packages

In [ ]:
%pip install -q "redisvl>=0.4.1" sentence-transformers pandas "redis>=5.2.0"

### Data/Index Preparation
 
In this section:

1. We prepare the data necessary for our hybrid search implementations by loading a collection of movies. Each movie object contains the following attributes:
    - `title`
    - `rating`
    - `description`
    - `genre`
 
2. We generate vector embeddings from the movie descriptions. This allows users to perform searches that not only rely on exact matches but also on semantic relevance, helping them find movies that align closely with their interests.

3. After preparing the data, we populate a search index with these movie records, enabling efficient querying based on both lexical and vector-based search techniques.

Running remotely or in collab? Run this cell to download the necessary dataset.

In [ ]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings and full text fields. **We need to have a Redis
instance available.**

#### Local Redis
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### Alternative Redis Access (Cloud, Docker, other)
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [1]:
import os
import warnings

warnings.filterwarnings('ignore')

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client, load data, generate embeddings

In [2]:
from redis import Redis

client = Redis.from_url(REDIS_URL)

In [3]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [4]:
from redisvl.utils.vectorize import HFTextVectorizer

# load model for embedding our movie descriptions
model = HFTextVectorizer('sentence-transformers/all-MiniLM-L6-v2')

In [5]:
movie_data = [
    {
        **movie,
        "description_vector": model.embed(movie["description"], as_buffer=True, dtype="float32")
    } for movie in movies
]

In [6]:
movie_data[:3]

[{'title': 'Explosive Pursuit',
  'genre': 'action',
  'rating': 7,
  'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
  'description_vector': b'\x9bf|=\x0e`\n;"\x92\xb7;<\xcb~\xbd\xfad\xce\xbb\xc3\x16J=V\xa7?=\xedv\x95<d\xfa\x06\xbe\x14Y\xcf=(\x07p=?\xdb\r\xbd\x95\xf2H\xbdje\xc6<E\xdfa=z8\x16\xbc\x00\xd4\x13<>\xaa\x1c=\xfd\xee\x89<\xbd\xb0-<\x82\xb2\x9f\xbc[\x0b\xc3\xbd\x98NR=xl\xf7\xbcN>\x17\xbe#\x12\x05\xb99u\xbf<\xb0\xe0b\xba\xd3\xa6\xa8\xbdx\xdc\xec\xbcRc%=\xe4\xe7r\xbb\x1eOG=?(\x85=o@\xa2\xbc2Z\xd0\xbdC%K\xbd\xb9\xed\x94\xbcR\xddH=\x92&F<\xc6*\xec<\x90\xd8\x8d\xbd\xcbZ\x98<\t\xa3\xa3=>g3\xbd&\xcd\xbd\xbd\x95$\xf7;\xfd\xf4z=\xfc\xb4\x8c=\x85\x0e\xc6\xbdnI\x90\xbdJ\x16\xbd;s\xe7\x0c\xbd 3\xc9\xbc\x85\xf8\xbb\xbc\xbf&u\xbb5\x8f\xca<\x05\x80J=\x0f\xaf*=\x8bOU\xbd\xc8\xf0\x95\xbc\x1d\x02\x19=)\xf4K<\xcb\xc2\t=F\x83\xac=\x9f\xd7\xb8\xbd\xf2\xb5\x9c\xbdB\x85\x18=\x96d&=-3\xf8<\xfa\xf7\x88<\x16v\xf2\xbb-=[\xbd\xf7\xac\xee\

### Define Redis index schema

Below, we build a schema that represents our movie objects.

In [7]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex


schema = IndexSchema.from_dict({
  "index": {
    "name": "movies",
    "prefix": "movie",
    "storage": "hash"
  },
  "fields": [
    { "name": "title", "type": "text" },
    { "name": "description", "type": "text" },
    { "name": "genre", "type": "tag", "attrs": {"sortable": True}},
    { "name": "rating", "type": "numeric", "attrs": {"sortable": True}},
    {
        "name": "description_vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float32"
        }
    }
  ]
})


index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

12:50:10 redisvl.index.index INFO   Index already exists, overwriting.


### Populate index

Load movie objects into Redis

In [8]:
index.load(movie_data)

['movie:0d8537e75af24af6b118f4629c2758a3',
 'movie:28c6547937a7402cba307cbe113c322f',
 'movie:d9bb23e83e1c41ad89f5de18ea5c65d6',
 'movie:b1d51e082d954d2eb00690885b601e76',
 'movie:54c806022aae415b9e1e23a110ca78d4',
 'movie:b81aad8ca262422cb80ba725b17afce4',
 'movie:54501580991a4e509194de51e9243159',
 'movie:1b184aed092e4a83a9b9ff15ae38b9b3',
 'movie:2b57e7ff370347528f0d1007bf7b9efd',
 'movie:ff0a1ca4f858447aab548bd9e12106cf',
 'movie:b0d7a84540f644099f411e7a5e2324c2',
 'movie:563bc43778494fe49ecb4884c20a0015',
 'movie:317ca39cce5e4817a153fa24821c83dc',
 'movie:ae506669d2fb47988398b6ea70c76a43',
 'movie:22c9dd9473104cde933a8d7918818fd5',
 'movie:dba67e0f8f4f45e38ba58533a7e70ec3',
 'movie:2f6abe9b28034db58e37d9b3cf4b0e96',
 'movie:64282c7d9f654e2d9c607a025ca25f2f',
 'movie:88b925781ac84dceb267ab92015450f1',
 'movie:8db639fc4fc44771b8e3285bf6a236e8']

# Hybrid Search Approaches

Now that our search index is populated and ready, we will build out a few different hybrid search techniques in Redis.

To start, we will build a few helper methods that we can reuse for each technique.

In [9]:
# Sample user query (can be changed for comparisons)
user_query = "action adventure movie with great fighting scenes, crime busting, superheroes, and magic"

First, we need a method to tokenize a user query into a full-text search string:

In [10]:
# list of stopwords to filter out noise from query string
stopwords = set([
    "a", "is", "the", "an", "and", "are", "as", "at", "be", "but", "by", "for",
    "if", "in", "into", "it", "no", "not", "of", "on", "or", "such", "that", "their",
    "then", "there", "these", "they", "this", "to", "was", "will", "with"
])

def tokenize_query(user_query: str) -> str:
    """Convert a raw user query to a redis full text query joined by ORs"""
    tokens = [token.strip().strip(",").lower() for token in user_query.split()]
    return " | ".join([token for token in tokens if token not in stopwords])

# Example
tokenize_query(user_query)

'action | adventure | movie | great | fighting | scenes | crime | busting | superheroes | magic'

Next, we need methods to create vector search and full-text search queries:

In [11]:
# Function to create a vector query using RedisVL helpers for ease of use
from redisvl.query import VectorQuery, FilterQuery
from redisvl.query.filter import Text
from redisvl.redis.utils import convert_bytes, make_dict


def make_vector_query(user_query: str, num_results: int, filters = None) -> VectorQuery:
    """Generate a Redis vector query given user query string."""
    vector = model.embed(user_query, as_buffer=True, dtype="float32")
    query = VectorQuery(
        vector=vector,
        vector_field_name="description_vector",
        num_results=num_results,
        return_fields=["title", "description"]
    )
    if filters:
        query.set_filter(filters)
    
    return query


def make_ft_query(text_field: str, user_query: str, num_results: int) -> FilterQuery:
    """Generate a Redis full-text query given a user query string."""
    return FilterQuery(
        filter_expression=f"~({Text(text_field) % tokenize_query(user_query)})",
        num_results=num_results,
        return_fields=["title", "description"],
        dialect=4,
    ).scorer("BM25").with_scores()

## 1. Linear Combination using Aggregation API

The goal of this technique is to calculate a weighted sum of the BM25 score for our provided text search and the cosine distance between vectors calculated via a KNN vector query. This is possible in Redis using the [aggregations API](https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/aggregations/), as of `Redis 7.4.x` (search version `2.10.5`), within a single database call.

In Redis, the aggregations api allow you the ability to group, sort, and transform your result data in the ways you might expect to be able to do with groupby and sums in other database paradigms.  


First, we build a base `VectorQuery` that runs a KNN-style vector search and test it below:

In [12]:
query = make_vector_query(user_query, num_results=3)

# Check standard vector search results
index.query(query)

[{'id': 'movie:dba67e0f8f4f45e38ba58533a7e70ec3',
  'vector_distance': '0.643690049648',
  'title': 'The Incredibles',
  'description': "A family of undercover superheroes, while trying to live the quiet suburban life, are forced into action to save the world. Bob Parr (Mr. Incredible) and his wife Helen (Elastigirl) were among the world's greatest crime fighters, but now they must assume civilian identities and retreat to the suburbs to live a 'normal' life with their three children. However, the family's desire to help the world pulls them back into action when they face a new and dangerous enemy."},
 {'id': 'movie:0d8537e75af24af6b118f4629c2758a3',
  'vector_distance': '0.668439269066',
  'title': 'Explosive Pursuit',
  'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.'},
 {'id': 'movie:b81aad8ca262422cb80ba725b17afce4',
  'vector_distance': '0.698122382164',
  'title': 'Mad Max: Fury Road',
  'description': "In a post-a

Next, we add a full-text search predicate using RedisVL helpers and our user-query tokenizer:

In [13]:
base_full_text_query = str(Text("description") % tokenize_query(user_query))

# Add the optional flag, "~", so that this doesn't also act as a strict text filter
full_text_query = f"(~{base_full_text_query})"


# Add full-text predicate to the vector query 
query.set_filter(full_text_query)
query.query_string()

'(~@description:(action | adventure | movie | great | fighting | scenes | crime | busting | superheroes | magic))=>[KNN 3 @description_vector $vector AS vector_distance]'

**The query string above combines both full-text search and a vector search.** This will be passed to the aggregation API to combine using a simple weighted sum of scores before a final sort and truncation.

Note: for the following query to work `redis-py >= 5.2.0`

In [14]:
from typing import Any, Dict, List
from redis.commands.search.aggregation import AggregateRequest, Desc

# Build the aggregation request
req = (
    AggregateRequest(query.query_string())
        .scorer("BM25")
        .add_scores()
        .apply(cosine_similarity="(2 - @vector_distance)/2", bm25_score="@__score")
        .apply(hybrid_score=f"0.3*@bm25_score + 0.7*@cosine_similarity")
        .load("title", "description", "cosine_similarity", "bm25_score", "hybrid_score")
        .sort_by(Desc("@hybrid_score"), max=3)
        .dialect(4)
)

# Run the query
res = index.aggregate(req, query_params={'vector': query._vector})

# Perform output parsing
[make_dict(row) for row in convert_bytes(res.rows)]


[{'vector_distance': '0.643690049648',
  '__score': '0.968066079387',
  'title': 'The Incredibles',
  'description': "A family of undercover superheroes, while trying to live the quiet suburban life, are forced into action to save the world. Bob Parr (Mr. Incredible) and his wife Helen (Elastigirl) were among the world's greatest crime fighters, but now they must assume civilian identities and retreat to the suburbs to live a 'normal' life with their three children. However, the family's desire to help the world pulls them back into action when they face a new and dangerous enemy.",
  'cosine_similarity': '0.678154975176',
  'bm25_score': '0.968066079387',
  'hybrid_score': '0.765128306439'},
 {'vector_distance': '0.668439269066',
  '__score': '0',
  'title': 'Explosive Pursuit',
  'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
  'cosine_similarity': '0.665780365467',
  'bm25_score': '0',
  'hybrid_score': '0.466046255

Notes on aggregate query syntax 
- `.scorer`: specifies the scoring function to use BM25 in this case
    - [see docs](https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/scoring/) for all available scorers
- `.add_scores`: adds the scores to the result
- `.apply`: algebraic operations that can be customized for your use case
- `.load`: specifies fields to return - all in this case.
- `.sort_by`: sort the output based on the hybrid score and yield top 5 results
- `.dialect`: specifies the query dialect to use.

Now we will define a function to do the entire operation start to finish for simplicity.

In [15]:
def linear_combo(user_query: str, alpha: float, num_results: int = 3) -> List[Dict[str, Any]]:
    # Add the optional flag, "~", so that this doesn't also act as a strict text filter
    text = f"(~{Text('description') % tokenize_query(user_query)})"

    # Build vector query
    query = make_vector_query(user_query, num_results=num_results, filters=text)
    
    # Build aggregation
    req = (
        AggregateRequest(query.query_string())
            .scorer("BM25")
            .add_scores()
            .apply(cosine_similarity="(2 - @vector_distance)/2", bm25_score="@__score")
            .apply(hybrid_score=f"{1-alpha}*@bm25_score + {alpha}*@cosine_similarity")
            .sort_by(Desc("@hybrid_score"), max=num_results)
            .load("title", "description", "cosine_similarity", "bm25_score", "hybrid_score")
            .dialect(4)
    )

    # Run the query
    res = index.aggregate(req, query_params={'vector': query._vector})

    # Perform output parsing
    if res:
        movies = [make_dict(row) for row in convert_bytes(res.rows)]
        return [(movie["title"], movie["hybrid_score"]) for movie in movies]

In [16]:
# Test it out

# 70% of the hybrid search score based on cosine similarity
linear_combo(user_query, alpha=0.7, num_results=6)

[('The Incredibles', '0.765128306439'),
 ('Explosive Pursuit', '0.466046255827'),
 ('Mad Max: Fury Road', '0.455657166243'),
 ('The Dark Knight', '0.452280691266'),
 ('Despicable Me', '0.448826777935'),
 ('Inception', '0.434456580877')]

## 2. Client-side fusion with RRF

Instead of relying on document scores like cosine similarity and BM25/TFIDF, we can fetch items and focus on their rank. This rank can be utilized to create a new ranking metric known as [Reciprocal Rank Fusion (RRF)](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf). RRF is powerful because it can handle ranked lists of different length, scores of different scales, and other complexities.

Although Redis does not currently support RRF natively, we can easily implement it on the client side.

In [17]:
def fuse_rankings_rrf(*ranked_lists, weights=None, k=60):
    """
    Perform Weighted Reciprocal Rank Fusion on N number of ordered lists.
    """
    item_scores = {}
    
    if weights is None:
        weights = [1.0] * len(ranked_lists)
    else:
        assert len(weights) == len(ranked_lists), "Number of weights must match number of ranked lists"
        assert all(0 <= w <= 1 for w in weights), "Weights must be between 0 and 1"
    
    for ranked_list, weight in zip(ranked_lists, weights):
        for rank, item in enumerate(ranked_list, start=1):
            if item not in item_scores:
                item_scores[item] = 0
            item_scores[item] += weight * (1 / (rank + k))
    
    # Sort items by their weighted RRF scores in descending order
    return sorted(item_scores.items(), key=lambda x: x[1], reverse=True)

In [18]:
# Below is a simple example of RRF over a few lists of numbers
fuse_rankings_rrf([1, 2, 3], [2, 4, 6, 7, 8], [5, 6, 1, 2])

[(2, 0.04814747488101534),
 (1, 0.032266458495966696),
 (6, 0.03200204813108039),
 (5, 0.01639344262295082),
 (4, 0.016129032258064516),
 (3, 0.015873015873015872),
 (7, 0.015625),
 (8, 0.015384615384615385)]

In [19]:
def weighted_rrf(
    user_query: str,
    alpha: float = 0.5,
    num_results: int = 4,
    k: int = 60,
) -> List[Dict[str, Any]]:
    """Implemented client-side RRF after querying from Redis."""
    # Create the vector query
    vector_query = make_vector_query(user_query, num_results=len(movie_data))

    # Create the full-text query
    full_text_query = make_ft_query("description", user_query, num_results=len(movie_data))

    # Run queries individually
    vector_query_results = index.query(vector_query)
    full_text_query_results = index.query(full_text_query)

    # Extract titles from results
    vector_titles = [movie["title"] for movie in vector_query_results]
    full_text_titles = [movie["title"] for movie in full_text_query_results]

    # Perform weighted RRF
    return fuse_rankings_rrf(vector_titles, full_text_titles, weights=[alpha, 1-alpha], k=k)[:num_results]

In [20]:
# Test it out!
weighted_rrf(user_query, num_results=6)

[('The Incredibles', 0.016009221311475412),
 ('Explosive Pursuit', 0.01575682382133995),
 ('Mad Max: Fury Road', 0.015079365079365078),
 ('Finding Nemo', 0.015008960573476702),
 ('Fast & Furious 9', 0.014925373134328358),
 ('The Dark Knight', 0.014854753521126762)]

But say we want to give more weight to the vector search rankings in this case to boost semantic similarities contribution to the final rank:

In [21]:
weighted_rrf(user_query, alpha=0.7, num_results=6)

[('The Incredibles', 0.016162909836065574),
 ('Explosive Pursuit', 0.015905707196029777),
 ('Mad Max: Fury Road', 0.015396825396825395),
 ('The Dark Knight', 0.015162852112676057),
 ('Fast & Furious 9', 0.014925373134328356),
 ('Inception', 0.014715649647156496)]

## 3. Client-side reranking

An alternative approach to RRF is to simply use an external reranker to order the final recommendations. RedisVL has built-in integrations to a few popular reranking modules.

In [22]:
from redisvl.utils.rerank import HFCrossEncoderReranker

# Load the ms marco MiniLM cross encoder model from huggingface
reranker = HFCrossEncoderReranker("cross-encoder/ms-marco-MiniLM-L-6-v2")


def rerank(
    user_query: str,
    num_results: int = 4,
) -> List[Dict[str, Any]]:
    """Rerank the candidates based on the user query with an external model/module."""
    # Create the vector query
    vector_query = make_vector_query(user_query, num_results=num_results)

    # Create the full-text query
    full_text_query = make_ft_query("description", user_query, num_results=num_results)

    # Run queries individually
    vector_query_results = index.query(vector_query)
    full_text_query_results = index.query(full_text_query)

    # Assemble list of potential movie candidates with their IDs
    movie_map = {}
    for movie in vector_query_results + full_text_query_results:
        candidate = f"Title: {movie['title']}. Description: {movie['description']}"
        if candidate not in movie_map:
            movie_map[candidate] = movie

    # Rerank candidates
    reranked_movies, scores = reranker.rank(
        query=user_query,
        docs=list(movie_map.keys()),
        limit=num_results,
        return_score=True
    )

    # Fetch full movie objects for the reranked results
    return [
        (movie_map[movie['content']]["title"], score)
        for movie, score in zip(reranked_movies, scores)
    ]


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [23]:
# Test it out!
rerank(user_query, num_results=6)

[('The Incredibles', -0.45268189907073975),
 ('The Dark Knight', -7.411877632141113),
 ('Explosive Pursuit', -8.751346588134766),
 ('Mad Max: Fury Road', -7.049145698547363),
 ('Aladdin', -9.638406753540039),
 ('Despicable Me', -9.797615051269531)]

This technique is certainly much slower than simple RRF as it's running an additional cross-encoder model to rerank the results. This can be fairly computationally expensive, but tunable with enough clarity on the use case and focus (how many items to retrieve? how many items to rerank? model accleration via GPU?)

## Comparing Approaches

While each approach has strengths and weaknesses, it's important to understand that each might work better in some use cases than others. Below we will run through a sample of user queries and generate matches for each using different hybrid search techniques.

In [24]:
movie_user_queries = [
    "I'm in the mood for a high-rated action movie with a complex plot",
    "What's a funny animated film about unlikely friendships?",
    "Any movies featuring superheroes or extraordinary abilities", 
    "I want to watch a thrilling movie with spies or secret agents",
    "Are there any comedies set in unusual locations or environments?",
    "Find me an action-packed movie with car chases or explosions",
    "What's a good family-friendly movie with talking animals?",
    "I'm looking for a film that combines action and mind-bending concepts",
    "Suggest a movie with a strong female lead character",
    "What are some movies that involve heists or elaborate plans?",
    "I need a feel-good movie about personal growth or transformation",
    "Are there any films that blend comedy with action elements?", 
    "Show me movies set in dystopian or post-apocalyptic worlds",
    "I'm interested in a movie with themes of revenge or justice",
    "What are some visually stunning movies with impressive special effects?"
]

In [25]:
import pandas as pd


rankings = pd.DataFrame()
rankings["queries"] = movie_user_queries

# First, add new columns to the DataFrame
rankings["hf-cross-encoder"] = ""
rankings["rrf"] = ""
rankings["linear-combo-bm25-cosine"] = ""

In [26]:
# Now iterate through the queries and add results
for i, user_query in enumerate(movie_user_queries):
    rankings.at[i, "hf-cross-encoder"] = rerank(user_query, num_results=4)
    rankings.at[i, "rrf"] = weighted_rrf(user_query, alpha=0.7, num_results=4)
    rankings.at[i, "linear-combo-bm25-cosine"] = linear_combo(user_query, alpha=0.7, num_results=4)

In [27]:
rankings.head()

,queries,hf-cross-encoder,rrf,linear-combo-bm25-cosine
0,I'm in the mood for a high-rated action movie ...,"[(Explosive Pursuit, -11.244140625), (Mad Max:...","[(The Incredibles, 0.016029143897996357), (Mad...","[(The Incredibles, 0.552392209158), (Despicabl..."
1,What's a funny animated film about unlikely fr...,"[(Despicable Me, -10.441911697387695), (The In...","[(Black Widow, 0.015625), (The Incredibles, 0....","[(The Incredibles, 0.454752022028), (Despicabl..."
2,Any movies featuring superheroes or extraordin...,"[(The Incredibles, -3.6648106575012207), (The ...","[(The Incredibles, 0.01639344262295082), (Mad ...","[(The Incredibles, 0.603234936448), (The Aveng..."
3,I want to watch a thrilling movie with spies o...,"[(The Incredibles, -10.843631744384766), (Expl...","[(Skyfall, 0.01631411951348493), (Explosive Pu...","[(Skyfall, 0.44384047389), (Despicable Me, 0.4..."
4,Are there any comedies set in unusual location...,"[(The Incredibles, -11.45376968383789), (Explo...","[(Madagascar, 0.015272878190495952), (Explosiv...","[(Madagascar, 0.442132177949), (Despicable Me,..."


In [28]:
rankings.loc[12].values

array(['Show me movies set in dystopian or post-apocalyptic worlds',
       list([('Mad Max: Fury Road', -3.490626335144043), ('Despicable Me', -11.051526069641113), ('The Incredibles', -11.315656661987305), ('Black Widow', -10.880638122558594)]),
       list([('Mad Max: Fury Road', 0.01602086438152012), ('Skyfall', 0.015607940446650124), ('The Incredibles', 0.015237691001697792), ('Black Widow', 0.01513526119402985)]),
       list([('Mad Max: Fury Road', '0.452238571644'), ('The Incredibles', '0.445061546564'), ('Madagascar', '0.41901564002'), ('Despicable Me', '0.416218408942')])],
      dtype=object)

# Wrap up
That's a wrap! Hopefully from this you were able to learn:
- How to implement simple vector search queries in Redis
- How to implement vector search queries with full-text filters
- How to implement hybrid search queries using the Redis aggregation API
- How to perform client-side fusion and reranking techniques